In [1]:
import os
import requests
import torch
from torch.nn import functional as F
torch.manual_seed(1337)

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-10-01 12:03:56--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-10-01 12:03:57 (24.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
import requests
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
# Send a GET request to fetch the file content
response = requests.get(url)
# Check if the request was successful
if response.status_code == 200:
    text_data = response.text
    #print(text_data)
else:
    print(f"Failed to fetch the file. Status code: {response.status_code}")

with open("input.txt", 'w', encoding = 'utf-8') as f:
  f.write(text_data)


In [6]:
with open("input.txt", 'r', encoding = 'utf-8') as f:
  text = f.read()

print(f"lenght of the character: {len(text)}")

lenght of the character: 1115394


In [7]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print('joint characters:',''.join(chars))
print('vocab size:',vocab_size)

joint characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [9]:
# mapping from chars to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }


def test(a):
  return a**2
test = lambda a : a**2
test(2)

encoder = lambda s: [stoi[ch] for ch in s]
decoder = lambda l: ''.join(itos[i] for i in l)
print(encoder("hii there"))
print(decoder(encoder("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [10]:
import torch
torch.long

torch.int64

In [11]:
import torch
data = torch.tensor(encoder(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [12]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

block_size = 8
train_data[:block_size+1]

x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [13]:
train_data[1:block_size+1]

tensor([47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [15]:
print(f"train data shape{train_data.shape}")
print(f"batch shape{xb.shape}")
print(f"block shape{yb.shape}")

train data shapetorch.Size([1003854])
batch shapetorch.Size([4, 8])
block shapetorch.Size([4, 8])


In [ ]:
# self attention :
# till now in bygram model we have seen that 
# the future chatacter only depends on the previous character thats it.
# but if i want to give that future word the reference of all past chars
# not only the previous one '
# than one thing which i can do is to take weighted sum of 
# all the past characters numeric representation (embed vector)


In [16]:
 # Lets say we have below B,T, C 

torch.manual_seed(1337)
B,T,C = 4,8,2    # batch, Time, Channels  ( batch, sequence lenth, embd vector dim)
x = torch.randn(B,T,C)
x = torch.randint(2, 5, (4, 8, 2), dtype= torch.float)
x.shape, x[0]

(torch.Size([4, 8, 2]),
 tensor([[4., 2.],
         [2., 2.],
         [3., 2.],
         [4., 2.],
         [3., 3.],
         [2., 4.],
         [2., 4.],
         [2., 2.]]))

In [17]:
# We want x[b,t] = mean_{i<=t} x[b,i]
# basicaly we are taking the mean of previous numerics 
# for any perticcular time t
# we are taking the mean of previous times <t 



xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [18]:
# actual first example of batch size 4
# having time seq of 8 and each having 
# 2 dim  embd vec representation
x[0]

tensor([[4., 2.],
        [2., 2.],
        [3., 2.],
        [4., 2.],
        [3., 3.],
        [2., 4.],
        [2., 4.],
        [2., 2.]])

In [19]:
# below is the weighted mean for all the time steps
# 1st t is 4,2
# 2nd t is 4+2/2 = 3, 2+2/2 = 2--> (3,2)
# 3rd t is 4+2+3/3 = 3, 2+2+2/3 = 3-->(3,3)
# like wisw for all the 8 time steps
xbow[0]

tensor([[4.0000, 2.0000],
        [3.0000, 2.0000],
        [3.0000, 2.0000],
        [3.2500, 2.0000],
        [3.2000, 2.2000],
        [3.0000, 2.5000],
        [2.8571, 2.7143],
        [2.7500, 2.6250]])

In [20]:
# belw is the example of doing the weigted sum for previous time steps
# with metrix multiplication
# using lower tringular identiy metrix

# below is only doing the sum of preivous time steps values
# not the avg


# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
# a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [21]:
# below example ius doing the avg of the previous values

# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [37]:
# same thing we can do it in our peoblem statement

# version 1: using matrix multiply for a weighted aggregation

wei = torch.tril(torch.ones(T,T))
print(f"{wei=}")
wei = wei/wei.sum(1, keepdim=True)
print(f"{wei=}")

print(f"{wei.shape = }") #shape (T,T)
print(F"{x.shape=}")

xbow2 = wei @ x 
# (8,8)@(4,8,2)-> (4,8,2)
# (T,T) will get mul with each batch of size (T,C)
# and will generate (B,T,C) metrix

# internaly torch do broadcasting
# it takes (T,T) metrix and add Batch size to it
# make it (B,T,T) and the myultiply it with (B,T,C)
# so now it will do metrix multiplication
# of metrix of size (T,T) and (T,C) each in batch size B


print(f"{x[0]=}")
print(f"{xbow2[0]=}")
print(f"{xbow2.shape}")

wei=tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
wei=tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
wei.shape = torch.Size([8, 8])
x.shape=torch.Size([4, 8, 2])
x

In [ ]:
# Broadcasting and Batched Matrix Multiplication
# When PyTorch encounters tensors with more than two dimensions (like `(2, 4, 2)`), it treats the additional dimensions as batches of matrices.
# - **Tensor 1 (shape `(4, 4)`)**: This is a simple 2D matrix.
# - **Tensor 2 (shape `(2, 4, 2)`)**: This is a batch of 2 matrices, each of shape `(4, 2)`.
# When you try to multiply these using the `@` operator, PyTorch automatically **broadcasts** the single matrix `(4, 4)` across the two matrices in the batch `(2, 4, 2)`. So, it will treat the multiplication as if you are multiplying the matrix `(4, 4)` with each of the two matrices in the batch of shape `(4, 2)`.
# Mathematically, what happens is:
# 1. **First step**: PyTorch broadcasts the matrix of shape `(4, 4)` across the batch dimension of `tensor2`, so it implicitly creates two copies of the `(4, 4)` matrix. 
#    Now you have:
#    - One matrix of shape `(4, 4)` being multiplied by a matrix of shape `(4, 2)` from the batch.
#    - This happens for each batch item, so you are doing two separate matrix multiplications.
# 2. **Second step**: PyTorch performs the matrix multiplication for each item in the batch:
#    - The first `(4, 4)` matrix multiplies with the first `(4, 2)` matrix, resulting in a `(4, 2)` output.
#    - The second `(4, 4)` matrix multiplies with the second `(4, 2)` matrix, resulting in another `(4, 2)` output.
# 3. **Final result**: After doing the matrix multiplication for each item in the batch, you get a final output tensor of shape `(2, 4, 4)` (two matrices of shape `(4, 4)`).


In [74]:
# Broadcasting allows PyTorch to automatically 
# expand the dimensions of tensors so they can 
# have compatible shapes for element-wise operations 
# like addition. The smaller tensor 
# (the one with fewer dimensions) gets automatically 
# expanded across the larger tensor's 
# dimensions if possible.

b = torch.randint(0,10,(3,2)).float()
c = torch.randint(1,2,(1,2)).float()
e = torch.randint(1,2,(3,1)).float()
d = torch.randint(1,2,(2,)).float()

# Tensor 1D ( vector)
# tensor 2D ( metrix)
# tensor 3D ( stack of metrix) (batches oif metrix)
    # (batch, Time, channel)
    # (batch, seq length, embd vector dim)
# tensor 4D 
    # (batch, hight, widh, channle)


# c: A 2D tensor (matrix) with shape (1, 2), 
# meaning 1 row and 2 columns.
# d: A 1D tensor (vector) with shape (2,), 
# meaning it has only 2 elements in a 
# single dimension, without explicit rows or columns.

# c has two elements arranged in a row: 
# [[1., 1.]] (2D).
# d has two elements in a single dimension: 
# [1., 1.] (1D).

print(f"{b=}")
print(f"{b.shape=}")
print(f"{c=}")
print(f"{c.shape=}")
print(f"{e=}")
print(f"{e.shape=}")
print(f"{d=}")
print(f"{d.shape=}")

print(f"-----------------------")
print(f"{b+c=}")
print(f"{b+e=}")
print(f"{b+d=}")


b=tensor([[0., 8.],
        [2., 4.],
        [0., 1.]])
b.shape=torch.Size([3, 2])
c=tensor([[1., 1.]])
c.shape=torch.Size([1, 2])
e=tensor([[1.],
        [1.],
        [1.]])
e.shape=torch.Size([3, 1])
d=tensor([1., 1.])
d.shape=torch.Size([2])
-----------------------
b+c=tensor([[1., 9.],
        [3., 5.],
        [1., 2.]])
b+e=tensor([[1., 9.],
        [3., 5.],
        [1., 2.]])
b+d=tensor([[1., 9.],
        [3., 5.],
        [1., 2.]])


In [91]:
# version 2: use Softmax

tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))

weim = wei.masked_fill(tril == 0, float('-inf'))
weis = F.softmax(weim, dim= -1)
# sof = e^x/sum (e^x1 + ....)
xbow3 = weis @ x


print(f"{tril=}")
print(f"{wei=}")
print(f"{weim=}")
print(f"{weis=}")

print(f"-----------------------------------------")
print(f"{x[0]=}")
print(f"{xbow3[0]=}")

tril=tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
wei=tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
weim=tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
      

In [95]:
# comparision of all the three method
# of doing same thing

# version 1: using loop
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)
#---------------------------------------------------
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)

print(torch.allclose(xbow, xbow2))

#--------------------------------------
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x

print(torch.allclose(xbow, xbow3))


True
True


In [ ]:
# out of the above three version 
# we will use the last one

# elf-attention!
#-------------------
#--------------------

# in self attention the weight matrix wei of zeors will 
# get replaced by the metrix multiplication of query @ key.transpose()
# wei =  query @ key.transpose()

# now the value of wei will depend on query and key
# and these query and keys value depens on the 
# embedding vectors corros to each word in embedd matrix
# so as the optimisation goes on emedding metrix updates its vectors
# corrospond to each token.
# so the query key updaets accordingly 
# which will again contribute in the 
# calculation of weighted avg of past time spteps
# it will fluctuate the weight corrosponds to the perticular char in the past time step
#



# role of softmax
# convert it in prob
# exponent gives the near 0 values for the extream -ive values 
# and at the same time it will convert
# all the -values to + value
# so the numeric values will be in good shape

# mask used
# so that future cant comm with past



In [ ]:

# role of softmax
# convert it in prob
# exponent gives the near 0 values for the extream -ive values 
# and at the same time it will convert
# all the -values to + value
# so the numeric values will be in good shape

# mask used
# so that future cant comm with past



In [ ]:
# we can think it as
# lets [token1,token2,token3,...]
# so for any token 
# we are calculating the weighted avg of past tokens
# considening the contribution of any past token to that token 
# which will decieded by the weight metrix wei
# in the optimisation process


In [100]:
# self attention :
# when the query key and values are from the same source
# cross attention :
# when query commimg from different sounrce 
# and key values are from different source

# encode :
# here we dont use mask
# all the tokens interact with each other
# use self attention


# decode :
# here we use cross attention 
# bxz key value comes from encoder side
# query comes from decoder 
# also use masked self attention

# GPT:
# has masked self attention only
# not cross attention

# 2017 attention all need paaper
# attention ( Q,K,V) = (softmax(QK.T)/root(dk))*v
# dk = head size (16)

# we are multpliying root(dk)
# to preserve the variance of wei
# if not mul by it var will increase to aprox head size

# why its import
# bcz at the intialization wei has to be farely diffused
# if the wei values are fairly diffeyused around near zero than
# out of soft max will be ok
# if the wei values are extream 
# than oyut of softmax will be tend to onhot encoding
# out sofmax will become peeky

head_size = 16
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1)
print(f"{k.shape=}")
print(f"{q.shape=}")
print(f"{k.var()=}")
print(f"{q.var()=}")
print(f"{wei.var()=}")


k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1) * head_size**-0.5
print(f"---------------------------")
print(f"{k.var()=}")
print(f"{q.var()=}")
print(f"{wei.var()=}")

k.shape=torch.Size([4, 8, 16])
q.shape=torch.Size([4, 8, 16])
k.var()=tensor(1.0204)
q.var()=tensor(1.0632)
wei.var()=tensor(14.6308)
---------------------------
k.var()=tensor(0.9891)
q.var()=tensor(1.0652)
wei.var()=tensor(1.0078)


In [108]:
print(torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5]), dim = -1))
print(torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5])*3, dim = -1))
print(torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5])*8, dim = -1))

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
tensor([0.1438, 0.0585, 0.2620, 0.0585, 0.4773])
tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])


In [6]:
ix = torch.randint(6 - 3, (4,))
ix.shape
ix

tensor([2, 1, 0, 1])

In [24]:
data = [i for i in range(20)]

ix = torch.randint(20-3, (4,))
ix

tensor([15, 15, 12, 10])

In [17]:
x = [data[i:i+3] for i in ix]
y = [data[i+1:i+3+1] for i in ix]
print(x,y)

[[4, 5, 6], [2, 3, 4], [13, 14, 15], [6, 7, 8]] [[5, 6, 7], [3, 4, 5], [14, 15, 16], [7, 8, 9]]


In [27]:
torch.stack([torch.tensor(data[i:i+3]) for i in ix])

tensor([[15, 16, 17],
        [15, 16, 17],
        [12, 13, 14],
        [10, 11, 12]])

In [19]:
y

[[5, 6, 7], [3, 4, 5], [14, 15, 16], [7, 8, 9]]

In [29]:
import torch

random_tensor = torch.randn(4, 8, 10)
print(random_tensor.shape)


torch.Size([4, 8, 10])


In [36]:
l = torch.nn.Linear(10, 200)
l2 = torch.nn.Linear(200,10 )
l2(l(random_tensor)).shape

torch.Size([4, 8, 10])

In [37]:
l = torch.nn.Linear(10*8, 200)
l2 = torch.nn.Linear(200, 10*8)
l(random_tensor.reshape(4,10*8)).shape

torch.Size([4, 200])

In [38]:
l2(l(random_tensor.reshape(4,10*8))).shape

torch.Size([4, 80])

In [41]:
import torch

a1 = torch.randn(4, 8, 5)
print(a1.shape)
a2 = torch.randn(4, 8, 5)
print(a2.shape)

torch.Size([4, 8, 5])
torch.Size([4, 8, 5])


In [45]:
torch.concat([a1, a2], dim = -1).shape

torch.Size([4, 8, 10])

In [47]:

a1 = torch.randn(4, 9)
print(a1.shape)

torch.Size([4, 9])


In [50]:
a1[:, -3:].shape

torch.Size([4, 3])

In [51]:
a1

tensor([[-0.3912,  0.7407, -0.0651,  0.9849, -0.6156,  0.0040,  0.4691, -0.5427,
         -0.7073],
        [ 0.7824, -0.2396,  1.4771,  0.2353,  0.2687, -0.0293,  0.0441, -0.1262,
         -1.0035],
        [ 0.5595, -0.4299, -1.0468, -0.2405, -0.1348,  2.1035, -0.8706,  0.4621,
          0.2323],
        [ 0.9974, -0.4518, -0.2659, -0.5539, -2.6855, -1.5831, -1.0538, -0.1206,
          0.1416]])

In [52]:
a1[:, -3:]

tensor([[ 0.4691, -0.5427, -0.7073],
        [ 0.0441, -0.1262, -1.0035],
        [-0.8706,  0.4621,  0.2323],
        [-1.0538, -0.1206,  0.1416]])

In [60]:
a1 = torch.randn(1, 8, 2)
print(a1.shape)

torch.Size([1, 8, 2])


In [61]:
a1[:, -1, :].shape

torch.Size([1, 2])

In [62]:
a1

tensor([[[-0.0259,  0.2584],
         [-0.0914,  0.1731],
         [ 0.3069, -1.6863],
         [-1.1862,  0.6586],
         [ 1.2723, -0.1566],
         [-0.3084, -0.1227],
         [-0.0108,  1.3207],
         [-0.6291, -0.1495]]])

In [63]:
a1[:, -1, :]

tensor([[-0.6291, -0.1495]])

In [ ]:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

In [67]:
a1 = torch.randn(4,1)
a1.shape, a1

(torch.Size([4, 1]),
 tensor([[ 1.3938],
         [ 0.5535],
         [ 0.3968],
         [-0.6233]]))

In [72]:
torch.concat([a1,a1], dim=1).shape

torch.Size([4, 2])

In [70]:
torch.cat([a1,a1],dim = 1).shape

torch.Size([4, 2])

In [76]:
torch.concat((a1,a1),dim = 1)

tensor([[ 1.3938,  1.3938],
        [ 0.5535,  0.5535],
        [ 0.3968,  0.3968],
        [-0.6233, -0.6233]])

In [78]:
context = torch.zeros((1,1), dtype = torch.long)

In [82]:
context.shape

torch.Size([1, 1])

In [88]:
a1 = torch.randn(4,10)
a1.shape, a1

(torch.Size([4, 10]),
 tensor([[-1.1420,  1.1814,  0.3576, -1.8297,  1.4613,  0.8432,  0.1185, -0.2978,
          -2.2559,  1.3797],
         [-0.5352, -1.2428, -1.2278, -0.4866, -0.8216, -0.7622, -0.2490,  0.1305,
          -0.1554, -0.2390],
         [-0.5165,  0.5301,  0.7900, -0.2255,  0.2253,  2.3023,  0.5821,  0.5180,
          -0.1326,  1.0209],
         [-1.3290,  0.0755,  1.2942,  1.6286, -0.4758,  0.7090, -1.0108, -0.2441,
          -1.1692, -0.1003]]))

In [89]:
a1[0].tolist()

[-1.142049789428711,
 1.181437373161316,
 0.357624888420105,
 -1.8296600580215454,
 1.461300015449524,
 0.8432005643844604,
 0.11845407634973526,
 -0.29782533645629883,
 -2.255922317504883,
 1.3797030448913574]

In [91]:
torch.zeros(19).shape

torch.Size([19])